In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
import pyspark.sql.functions as f

In [ ]:
spark = SparkSession.builder \
    .appName("improve-enrich-data") \
    .getOrCreate()

In [ ]:
# Definir o schema dos dados na leitura
schema = StructType([
    StructField("invoice_no", StringType(), True),
    StructField("invoice_date", StringType(), True), # observe que é um tipo string para uma coluna de datetime ;)
    StructField("customer_id", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("category", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("selling_price_per_unit", DoubleType(), True),
    StructField("cost_price_per_unit", DoubleType(), True),
    StructField("payment_method", StringType(), True),
    StructField("region", StringType(), True),
    StructField("state", StringType(), True),
    StructField("shopping_mall", StringType(), True)
])

In [ ]:
'''
Sobre o Conjunto de Dados
Visão Geral

Este conjunto de dados é uma criação fictícia, projetada especificamente para ajudar analistas de 
dados a praticar análise exploratória de dados e visualização de dados. 
Ele contém informações sobre pedidos feitos por clientes em um aplicativo de entrega de produtos de 
supermercado.
'''
df = spark.read.csv("datasource/Different_stores_dataset.csv", header=True, schema=schema)

In [ ]:
df.show(n=1, truncate=False, vertical=True)

In [ ]:
from ydata_profiling import ProfileReport
'''
Esta linha importa a classe ProfileReport do módulo ydata_profiling que é uma biblioteca para análise exploratória de dados. 
Seu objetivo é gerar relatórios de perfil de conjuntos de dados, oferecendo uma visão geral abrangente dos dados.
'''
report = ProfileReport(
    df,
    title='Different stores dataset',       
    dataset={
        "description": "Dados fictícios sobre pedidos feitos por clientes em um aplicativo de entrega de produtos de supermercado",
        "copyright_holder": "Kushal Kamra",
        "url": "https://www.kaggle.com/datasets/kzmontage/sales-from-different-stores",
    },
    variables={
        "descriptions": {
            "invoice_no": "Número da fatura associado a cada transação.",
            "customer_id": "Identificador único para cada cliente.",
            "gender": "Gênero do cliente (presumido ser binário: masculino/feminino).",
            "age": "Idade do cliente.",
            "category": "Categoria do produto associada à transação.",
            "quantity": "Quantidade de produtos comprados em cada transação.",
            "selling_price_per_unit": "Preço de venda por unidade do produto.",
            "cost_price_per_unit": "Preço de custo por unidade do produto.",
            "payment_method": "Método de pagamento usado na transação.",
            "region": "Região geográfica associada à transação.",
            "state": "Estado onde a transação ocorreu.",
            "shopping_mall": "Shopping center onde ocorreu a transação.",
        }
    },
    infer_dtypes=False,
    interactions=None,
    missing_diagrams=None,
    correlations={"pearson": {"calculate": True}}
)

In [ ]:
report

In [ ]:
# Limpeza e padronização dos dados: Vamos melhorar o schema dos dados modificando o _Tipo_ da coluna `invoice_date` de String para Timestamp. 
df = df.withColumn("invoice_datetime", f.to_timestamp(df.invoice_date, 'MM/dd/yyyy H:mm'))

In [ ]:
# Padronização de Dados: Criar uma coluna extra invoice_date do _Tipo_ String no formato 'dd/MM/yyyy' para contrução de partição 
# melhorando o desempenho de consultas com data.
df = df.withColumn("invoice_date", f.date_format("invoice_datetime", 'dd/MM/yyyy'))

In [ ]:
df.select("invoice_datetime", "invoice_date").show()

In [ ]:
df.createOrReplaceTempView("stores")

In [ ]:
spark.sql(
    "SELECT * FROM stores"
).show(n=1, vertical=True)

In [ ]:
'''
1. Melhoramento e Enriquecimento de Dados:
    Você poderia enriquecer esses dados adicionando informações de rentabilidade para cada 
    transação (calculando a diferença entre selling_price_per_unit e cost_price_per_unit
    multiplicada pela quantity).
'''

In [ ]:
''' 
2. Eliminação de Dados Irrelevantes:
    Eliminar dados irrelevantes significa remover informações que não contribuem para a análise ou 
    objetivo específico.

    Se o objetivo for analisar apenas as vendas de produtos, campos como invoice_no e customer_id 
    podem ser considerados irrelevantes e, portanto, removidos.
    Em análises focadas em tendências, detalhes como shopping_mall podem ser descartados.
'''

In [ ]:
'''
3. Granulação e Agregação:
    A granulação e agregação envolvem resumir ou combinar dados para um nível de detalhe mais 
    alto ou mais baixo, dependendo da necessidade da análise.

    3.1 Granulação: Filtrar dados para um nível mais detalhado, como todas as transações de um 
    cliente específico (customer_id).

    3.2 Agregação: Sumarizar dados, como calcular o total de vendas por category ou a média de 
    idade dos clientes (age) por gender.
'''

In [ ]:
'''
4. Consistência, Integridade, Concordância:
    Esses conceitos referem-se a garantir que os dados sejam precisos, confiáveis e 
    consistentes em todo o conjunto de dados.

    Verificar se todos os invoice_date estão no formato correto e se os valores de age 
    são razoáveis (por exemplo, não negativos ou excessivamente altos).
    Assegurar que selling_price_per_unit seja sempre maior que cost_price_per_unit, 
    pois o contrário indicaria uma inconsistência nos dados
'''

In [ ]:
'''
5. Duplicação e Redundância:
    Dados duplicados ou redundantes podem distorcer análises e devem ser removidos ou tratados.
    
    Procurar por registros duplicados (por exemplo, duas transações com o mesmo invoice_no).
    Identificar redundâncias, como se a combinação de region e state sempre leva ao 
    mesmo shopping_mall, indicando que um desses campos pode ser redundante.
'''